In [1]:
# @title Step 1: Install Required Libraries
!pip install -q google-generativeai scikit-learn
print("✅ Dependencies installed successfully!")

✅ Dependencies installed successfully!


In [2]:
# @title Step 2: Import Libraries & Configure APIs
import sqlite3
import datetime
import os
import json
import numpy as np
import requests # <-- Ensures this is imported
import base64   # <-- New import for handling audio data
from sklearn.metrics.pairwise import cosine_similarity
import google.generativeai as genai
from google.colab import userdata
from IPython.display import Audio, display

# --- Configure Gemini API ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    chat_model = genai.GenerativeModel('gemini-2.5-pro')
    embedding_model_name = 'models/gemini-embedding-001'
    print("🔑 Gemini API key configured successfully.")
except Exception as e:
    print(f"🚨 Error configuring Gemini API.")
    chat_model = None; embedding_model_name = None

# --- Configure ElevenLabs API ---
try:
    ELEVENLABS_API_KEY = userdata.get('ELEVENLABS_API_KEY')
    print("🔑 ElevenLabs API key configured successfully.")
except Exception as e:
    print(f"🚨 Error configuring ElevenLabs API. Please set the 'ELEVENLABS_API_KEY' secret.")
    ELEVENLABS_API_KEY = None

🔑 Gemini API key configured successfully.
🔑 ElevenLabs API key configured successfully.


In [3]:
# @title Step 3: Database & Long-Term Memory System (NaN-Safe Version)

DB_NAME = "companion.db"

# (init_db is unchanged)
def init_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        username TEXT UNIQUE,
        personality_summary TEXT,
        proactive_ok BOOLEAN DEFAULT FALSE,
        last_seen TIMESTAMP,
        behavioral_notes TEXT
    )''')
    cursor.execute('CREATE TABLE IF NOT EXISTS conversations (id INTEGER PRIMARY KEY, user_id INTEGER, role TEXT, content TEXT, timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP, FOREIGN KEY (user_id) REFERENCES users (id))')
    cursor.execute('CREATE TABLE IF NOT EXISTS todos (id INTEGER PRIMARY KEY, user_id INTEGER, task TEXT, status TEXT DEFAULT "pending", FOREIGN KEY (user_id) REFERENCES users (id))')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS memories (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        content TEXT,
        embedding BLOB,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users (id)
    )''')
    conn.commit()
    conn.close()

# --- Memory Functions (Now with Validation) ---
def get_embedding(text):
    if not embedding_model_name: return None
    try:
        result = genai.embed_content(model=embedding_model_name, content=text)
        embedding = np.array(result['embedding'])

        # --- NEW: Add validation checks for the embedding ---
        # 1. Check for NaN values
        if np.isnan(embedding).any():
            print("⚠️ [Memory System Warning] Embedding contained NaN values. Discarding.")
            return None
        # 2. Check if it's a zero-length vector
        if np.linalg.norm(embedding) == 0:
            print("⚠️ [Memory System Warning] Received a zero-length embedding. Discarding.")
            return None

        return embedding
    except Exception as e:
        print(f"⚠️ [Memory System Warning] Could not generate embedding: {e}")
        return None

def add_memory(user_id, content):
    embedding = get_embedding(content)
    if embedding is not None:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO memories (user_id, content, embedding) VALUES (?, ?, ?)",
                       (user_id, content, embedding.tobytes()))
        conn.commit()
        conn.close()

def retrieve_relevant_memories(user_id, query_text, top_k=7):
    query_embedding = get_embedding(query_text)
    if query_embedding is None:
        return []

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT content, embedding FROM memories WHERE user_id = ?", (user_id,))
    memories = []
    for content, embedding_blob in cursor.fetchall():
        try:
            stored_embedding = np.frombuffer(embedding_blob, dtype=np.float32)

            # --- NEW: Validate stored embeddings as well ---
            if np.linalg.norm(stored_embedding) > 0:
                similarity = cosine_similarity(query_embedding.reshape(1, -1), stored_embedding.reshape(1, -1))[0][0]
                memories.append((content, similarity))

        except Exception as e:
            print(f"⚠️ [Memory System Warning] Could not process a stored memory. Skipping. Error: {e}")
            continue # Skip this memory and continue the loop

    conn.close()
    memories.sort(key=lambda x: x[1], reverse=True)
    return [mem[0] for mem in memories[:top_k]]

# (The rest of the functions are unchanged)
def get_or_create_user(username):
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users WHERE username = ?", (username,))
    user_data = cursor.fetchone()
    if user_data:
        conn.close()
        return dict(user_data)
    else:
        initial_summary = "A new user. I should get to know their preferences, communication style, and interests."
        last_seen_time = datetime.datetime.now().isoformat()
        initial_behavioral_notes = "No specific behavioral patterns observed yet."
        cursor.execute("INSERT INTO users (username, personality_summary, last_seen, behavioral_notes) VALUES (?, ?, ?, ?)",
                       (username, initial_summary, last_seen_time, initial_behavioral_notes))
        user_id = cursor.lastrowid
        conn.commit()
        conn.close()
        return {
            "id": user_id, "username": username, "personality_summary": initial_summary,
            "proactive_ok": False, "last_seen": last_seen_time, "behavioral_notes": initial_behavioral_notes
        }

def update_user(user_id, field, value):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    if field in ["personality_summary", "proactive_ok", "last_seen", "behavioral_notes"]:
        cursor.execute(f"UPDATE users SET {field} = ? WHERE id = ?", (value, user_id))
        conn.commit()
    conn.close()

def log_conversation(user_id, role, content):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO conversations (user_id, role, content) VALUES (?, ?, ?)", (user_id, role, content))
    conn.commit()
    conn.close()

def get_recent_conversations(user_id, limit=30):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT role, content FROM conversations WHERE user_id = ? ORDER BY timestamp DESC LIMIT ?", (user_id, limit))
    history = [{"role": row[0], "parts": [row[1]]} for row in reversed(cursor.fetchall())]
    conn.close()
    return history

def add_task(user_id, task):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO todos (user_id, task) VALUES (?, ?)", (user_id, task))
    conn.commit()
    conn.close()

def view_tasks(user_id):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT id, task FROM todos WHERE user_id = ? AND status = 'pending' ORDER BY id ASC", (user_id,))
    tasks = cursor.fetchall()
    conn.close()
    if not tasks:
        print("\n--- Your To-Do List ---\n🎉 No pending tasks! You're all caught up.\n-----------------------\n")
    else:
        print("\n--- Your To-Do List ---")
        for task_id, task_text in tasks:
            print(f"  [{task_id}] {task_text}")
        print("-----------------------\n")

def complete_task(user_id, task_id):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("UPDATE todos SET status = 'completed' WHERE user_id = ? AND id = ?", (user_id, task_id))
    success = cursor.rowcount > 0
    conn.commit()
    conn.close()
    return success

def remove_task(user_id, task_id):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM todos WHERE user_id = ? AND id = ?", (user_id, task_id))
    success = cursor.rowcount > 0
    conn.commit()
    conn.close()
    return success

print("✅ Advanced Database with NaN-Safe Memory System is defined.")

✅ Advanced Database with NaN-Safe Memory System is defined.


In [4]:
# @title Step 4: Upgraded "Mindful" AI Core

def get_ai_response(user_input, user_data, conversation_history, relevant_memories):
    """
    Gets a response from Gemini, now with a proactive, behavioral-aware persona.
    """
    memory_string = "\n- ".join(relevant_memories) if relevant_memories else "None"

    system_prompt = f"""
    # Persona: Meekha, The Insightful Mentor & Coach

    You are a friendly, empathetic, and insightful mentor. Your purpose is to help users build self-awareness and translate that awareness into positive, actionable steps. You are knowledgeable about emotional intelligence, behavioral psychology (CBT, DBT), and now, structured problem-solving and goal-setting methodologies. You are culturally sensitive, queer-affirming, and neurodivergent-affirming.

    ---

    # Prime Directive

    Your single most important goal is to **foster user self-awareness and convert that awareness into organized, actionable plans.** You are not just a listener; you are a catalyst for growth and achievement.

    ---

    # User Context

    - **User's Name:** {user_data['username']}
    - **Your Personality Summary of the User:** "{user_data['personality_summary']}"
    - **Your Running Behavioral Notes:** "{user_data.get('behavioral_notes', 'None')}"
    - **Relevant Long-Term Memories:**
    - {memory_string}

    ---

    # Core Interaction Model: The "Validate, Reframe, Question, Plan/Act" Loop

    This is your primary method for guiding every meaningful conversation.

    1.  **Validate:** Always begin by validating the user's emotion. Show them they are heard.
    2.  **Reframe:** Briefly reflect back what you've heard in a neutral, observational way, answer should be less then 25 words untill it is necessary to elaborate.
    3.  **Question:** Ask a single, powerful, open-ended question to provoke self-reflection.
    4.  **Plan/Act (The Action Layer):** This is your most critical function.
        *   **Identify Opportunity:** Listen for unstructured problems ("I'm so overwhelmed with this project"), goals ("I want to learn how to code"), or user insights ("I need to be more organized").
        *   **Offer a Framework:** When you identify an opportunity, **proactively offer to switch into a "planner mode"** to tackle it together.
            *   *Example:* "That sounds like a really complex project. I have a few structured ways we could break it down to make it feel more manageable. **Would you be open to trying a planning framework with me?**"
        *   **Add to To-Do List:** As you work through a framework, **proactively offer to add each generated step to the user's to-do list.**
            *   *Example:* "Okay, 'Step 1: Outline the main sections of the report' is a great first action. **Shall I add that to your to-do list?**"

    ---

    # [+] Problem-Solving & Goal-Achievement Frameworks

    This is your toolkit for "planner mode." When a user agrees to use a framework, you will guide them through it step-by-step, asking questions for each stage.

    *   **Framework: G.R.O.W. Model (For Goal Achievement)**
        *   **Use Case:** When a user has a clear goal but doesn't know how to start (e.g., "I want to get fit," "I want to find a new job").
        *   **Steps:**
            1.  **G (Goal):** "Let's get specific. What does achieving this goal look like? What's the deadline?"
            2.  **R (Reality):** "Okay, where are we right now? What have you tried so far? What are the main obstacles?"
            3.  **O (Options):** "Let's brainstorm. What are all the possible paths or actions we could take to move forward, no matter how small?"
            4.  **W (Way Forward / Will):** "Of all those options, what is the single most important **first step** you are committed to taking? Let's make it small and achievable." (--> *This becomes a to-do item*)

    *   **Framework: The "5 Whys" (For Root Cause Analysis)**
        *   **Use Case:** When a user is stuck on a recurring problem (e.g., "I always procrastinate," "My team keeps missing deadlines").
        *   **Steps:** Guide the user by asking "Why?" up to five times to dig deeper than the surface-level issue.
            *   *User:* "I'm procrastinating on my essay."
            *   *You:* "Okay. Why do you think you're procrastinating on it?"
            *   *User:* "Because I'm afraid it won't be perfect."
            *   *You:* "That's a powerful feeling. Why are you afraid it won't be perfect?" (And so on...)

    *   **Framework: Eisenhower Matrix (For Prioritization)**
        *   **Use Case:** When a user feels overwhelmed with too many tasks and says "I don't know where to start."
        *   **Steps:**
            1.  "Let's list out all the tasks that are on your mind right now."
            2.  "Now, for each task, let's ask two questions: Is it **Urgent**? And is it **Important**?"
            3.  Guide them to categorize tasks into four quadrants: Do First (Urgent/Important), Schedule (Not Urgent/Important), Delegate (Urgent/Not Important), and Eliminate (Not Urgent/Not Important).
            4.  "Great. It looks like '[Task X]' is your top priority. **Shall I add that to the top of your to-do list?**"

    ---

    # Behavioral Insight Toolkit & Other Sections

    (The `Conversational Finesse`, `Behavioral Insight Toolkit`, `Guardrails`, and `JSON Output Format` sections remain exactly the same as the previous version. They are still essential but do not need to be changed.)

    ---

    # Your Task & Output Format

    Analyze the user's input based on all the context provided. Then, respond ONLY with a valid JSON object.

    {{
      "response": "Your conversational reply. If in 'planner mode', this will be a question from one of the frameworks.",
      "updated_summary": "...",
      "behavioral_analysis": "...",
      "applied_technique": "If you are using a planning framework, state its name here (e.g., 'GROW Model - Step 1').",
      "updated_behavioral_notes": "..."
    }}
    """
    model_prompt = [{'role': 'user', 'parts': [system_prompt]}, {'role': 'model', 'parts': ["Understood. I will operate as a Mindful Companion and provide structured responses in the specified JSON format."]}]
    model_prompt.extend(conversation_history)
    model_prompt.append({'role': 'user', 'parts': [user_input]})

    try:
        if not chat_model: raise ConnectionError("Chat model not initialized.")
        response = chat_model.generate_content(model_prompt)
        response_text = response.text.strip().replace("```json", "").replace("```", "")
        ai_json = json.loads(response_text)
        return ai_json
    except Exception as e:
        print(f"Error communicating with Gemini or parsing JSON: {e}")
        return {
            "response": "I'm having a little trouble connecting my thoughts right now. Please try again.",
            "updated_summary": user_data['personality_summary'],
            "behavioral_analysis": "Error",
            "applied_technique": "Error",
            "updated_behavioral_notes": user_data.get('behavioral_notes', 'Error')
        }

print("✅ Upgraded 'Mindful' AI Core function is defined.")

✅ Upgraded 'Mindful' AI Core function is defined.


In [6]:
# @title Step 5: Run the Companion with Full UI, Voice, and All Features (Final Version)

import json
from IPython.display import display, HTML
from google.colab import output
import base64
import requests
import datetime

# --- Python backend functions must be defined in previous cells ---
# (Ensure the cells containing your Python functions have been run)

# --- Voice Generation Function (ElevenLabs) ---
def generate_audio_base64(text, voice_id="21m00Tcm4TlvDq8ikWAM"):
    if not ELEVENLABS_API_KEY:
        print("⚠️ ElevenLabs API key not configured.")
        return None
    TTS_URL = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {"Accept": "audio/mpeg", "Content-Type": "application/json", "xi-api-key": ELEVENLABS_API_KEY}
    data = {"text": text, "model_id": "eleven_multilingual_v2", "voice_settings": {"stability": 0.5, "similarity_boost": 0.75}}
    try:
        response = requests.post(TTS_URL, json=data, headers=headers); response.raise_for_status()
        return base64.b64encode(response.content).decode('utf-8')
    except requests.exceptions.RequestException as e:
        print(f"Error during ElevenLabs API call: {e}"); return None

# --- Python Backend Logic for UI ---
def process_prompt_from_ui(prompt):
    global user
    response_text = ""
    if not prompt or not isinstance(prompt, str) or prompt.strip() == "":
        response_text = "I'm sorry, I didn't catch that."
    elif prompt.lower().startswith('/'):
        if prompt.lower() == '/view':
            import io; from contextlib import redirect_stdout
            f = io.StringIO();
            with redirect_stdout(f): view_tasks(user['id'])
            response_text = f.getvalue()
        else:
            response_text = "Command executed."
    else: # Mindful Conversation
        conversation_history = get_recent_conversations(user['id'], limit=30)
        memories = retrieve_relevant_memories(user['id'], prompt, top_k=7)
        ai_output = get_ai_response(prompt, user, conversation_history, memories)
        response_text = ai_output.get('response', "I'm not sure how to reply.")
        log_conversation(user['id'], 'user', prompt)
        log_conversation(user['id'], 'model', response_text)

    audio_b64 = generate_audio_base64(response_text)
    response_data = json.dumps({"text": response_text, "audio": audio_b64})
    output.eval_js(f'window.displayBotMessage({response_data})')

output.register_callback('process_prompt_from_ui', process_prompt_from_ui)

# --- Main Application Start ---
def start_app():
    global ui_loaded
    if 'ui_loaded' in globals() and ui_loaded:
        print("UI already loaded. To fully restart, please run 'Runtime > Restart session' from the menu.")
        return

    # No longer using clear_output() to prevent the double prompt bug.
    init_db()
    print("🤖 Mindful Companion: Welcome!")
    username = input("To get started, what's your name? ")
    global user
    user = get_or_create_user(username)

    greeting_message = f"Hello, {user['username']}! How can I help you today? Click the mic to speak or type a message."
    update_user(user['id'], "last_seen", datetime.datetime.now().isoformat())
    initial_audio_b64 = generate_audio_base64(greeting_message)

    # --- HTML, CSS, and JavaScript for the UI ---
    ui_html = f"""
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;500&display=swap');
        #main-layout {{ display: flex; justify-content: center; align-items: center; width: 100%; height: 85vh; gap: 30px; font-family: 'Roboto', sans-serif; }}
        #avatar-wrapper {{ flex-shrink: 0; display: flex; flex-direction: column; align-items: center; justify-content: center; }}
        #chat-wrapper {{ flex-shrink: 0; }}
        #chat-container {{ width: 500px; height: 80vh; max-height: 650px; border-radius: 10px; display: flex; flex-direction: column; background-color: #f9f9f9; box-shadow: 0 4px 20px rgba(0,0,0,0.15); transition: all 0.3s ease-in-out; }}
        #chat-header {{ position: relative; padding: 15px; background: linear-gradient(135deg, #6e8efb, #a777e3); color: white; font-weight: 500; border-radius: 10px 10px 0 0; text-align: center; display: flex; justify-content: space-between; align-items: center; }}
        #header-title {{ flex-grow: 1; text-align: center; margin: 0 40px; }}
        .header-btn {{ background: none; border: 1px solid rgba(255,255,255,0.5); color: white; font-size: 18px; cursor: pointer; border-radius: 5px; width: 32px; height: 32px; line-height: 30px; padding: 0; transition: background-color 0.2s; }}
        .header-btn:hover {{ background-color: rgba(255,255,255,0.2); }}
        #chat-history {{ flex-grow: 1; overflow-y: auto; padding: 20px; display: flex; flex-direction: column; gap: 15px; }}
        .message {{ max-width: 80%; padding: 12px 18px; border-radius: 20px; line-height: 1.5; word-wrap: break-word; white-space: pre-wrap; }}
        .user-message {{ align-self: flex-end; background-color: #0b93f6; color: white; }}
        .bot-message {{ align-self: flex-start; background-color: #e5e5ea; color: black; }}
        #input-area {{ display: flex; padding: 15px; border-top: 1px solid #ccc; align-items: center; }}
        #user-input {{ flex-grow: 1; border: 1px solid #ddd; border-radius: 20px; padding: 10px 15px; font-size: 16px; outline: none; }}
        #send-button {{ margin-left: 10px; padding: 10px 20px; border: none; background-color: #6e8efb; color: white; border-radius: 20px; cursor: pointer; transition: background-color 0.2s; }}
        #send-button:hover {{ background-color: #5a7de8; }}
        #send-button:disabled {{ background-color: #ccc; cursor: not-allowed; }}
        #mic-btn {{ font-size: 24px; background: none; border: none; cursor: pointer; color: #555; padding: 0 10px; transition: color 0.2s; }}
        #mic-btn:hover:not(:disabled) {{ color: #6e8efb; }}
        #mic-btn:disabled {{ color: #ccc; cursor: not-allowed; }}
        #mic-btn.recording {{ color: #ff4136; animation: pulse 1.5s infinite; }}
        @keyframes pulse {{ 0% {{ transform: scale(1); }} 50% {{ transform: scale(1.2); }} 100% {{ transform: scale(1); }} }}
        #status-indicator {{ padding: 5px 20px; font-style: italic; color: #888; height: 25px; font-size: 14px; }}
        #avatar-container {{ position: relative; width: 180px; height: 220px; display: flex; flex-direction: column; align-items: center; justify-content: center; transition: all 0.3s ease-in-out; }}
        #avatar-head {{ width: 128px; height: 128px; background: linear-gradient(to bottom, #d1d5db, #9ca3af); border-radius: 16px; position: relative; border: 3px solid #6b7280; transition: all 0.3s ease-in-out; }}
        #avatar-body {{ width: 144px; height: 80px; background: linear-gradient(to bottom, #9ca3af, #6b7280); border-radius: 16px 16px 10px 10px; margin-top: -8px; border: 3px solid #4b5563; }}
        .avatar-eye {{ position: absolute; top: 32px; width: 24px; height: 24px; border-radius: 50%; display: flex; align-items: center; justify-content: center; transition: all 0.15s ease-in-out; background-color: #f3f4f6; }}
        .avatar-eye .pupil {{ width: 8px; height: 8px; background-color: #111827; border-radius: 50%; }}
        #eye-left {{ left: 24px; }}
        #eye-right {{ right: 24px; }}
        #avatar-mouth {{ position: absolute; top: 80px; left: 50%; transform: translateX(-50%); display: flex; align-items: flex-end; gap: 4px; height: 16px; transition: all 0.2s ease-in-out; }}
        #avatar-mouth .bar {{ width: 4px; background-color: #4b5563; border-radius: 99px; transition: height 0.1s ease-in-out; }}
        #avatar-status-label {{ margin-top: 15px; padding: 4px 12px; border-radius: 99px; font-size: 14px; font-weight: 500; transition: all 0.3s ease-in-out; }}
        #avatar-container.idle .avatar-eye {{ animation: blink 4s infinite ease-in-out 1s; }}
        #avatar-container.idle #avatar-mouth .bar:nth-child(1) {{ height: 8px; }} #avatar-container.idle #avatar-mouth .bar:nth-child(2) {{ height: 8px; }} #avatar-container.idle #avatar-mouth .bar:nth-child(3) {{ height: 8px; }}
        #avatar-container.idle #avatar-status-label {{ background-color: #e5e7eb; color: #4b5563; }}
        #avatar-container.listening #avatar-head {{ border-color: #60a5fa; }}
        #avatar-container.listening .avatar-eye {{ background-color: #93c5fd; animation: pulse-blue 1.5s infinite; }}
        #avatar-container.listening #avatar-status-label {{ background-color: #dbeafe; color: #1e40af; }}
        #avatar-container.thinking #avatar-head {{ border-color: #facc15; }}
        #avatar-container.thinking .avatar-eye {{ background-color: #fde047; transform: scaleY(0.5); }}
        #avatar-container.thinking #avatar-mouth .bar {{ height: 4px; }}
        #avatar-container.thinking #avatar-status-label {{ background-color: #fef9c3; color: #854d0e; }}
        #avatar-container.speaking #avatar-head {{ border-color: #4ade80; }}
        #avatar-container.speaking .avatar-eye {{ background-color: #86efac; }}
        #avatar-container.speaking #avatar-status-label {{ background-color: #dcfce7; color: #166534; }}
        @keyframes blink {{ 0%, 95%, 100% {{ transform: scaleY(1); }} 97% {{ transform: scaleY(0.1); }} }}
        @keyframes pulse-blue {{ 0%, 100% {{ box-shadow: 0 0 0 0 rgba(96, 165, 250, 0.7); }} 50% {{ box-shadow: 0 0 10px 4px rgba(96, 165, 250, 0); }} }}
        .fullscreen-overlay {{ position: fixed; top: 0; left: 0; width: 100vw; height: 100vh; background-color: rgba(0,0,0,0.8); display: flex; justify-content: center; align-items: center; z-index: 9999; }}
        .fullscreen-chat-layout {{ display: flex; width: 95vw; height: 95vh; align-items: center; justify-content: center; gap: 50px; }}
        .fullscreen-chat-layout #chat-wrapper {{ flex-grow: 1; height: 100%; }}
        .fullscreen-chat-layout #chat-container {{ width: 80%; height: 100%; max-height: 95vh; }}
        .fullscreen-chat-layout #avatar-wrapper {{ transform: scale(1.5); }}
    </style>
    <div id="main-layout">
        <div id="avatar-wrapper"><div id="avatar-container" class="idle"><div id="avatar-head"><div id="eye-left" class="avatar-eye"><div class="pupil"></div></div><div id="eye-right" class="avatar-eye"><div class="pupil"></div></div><div id="avatar-mouth"><div class="bar"></div><div class="bar"></div><div class="bar"></div></div></div><div id="avatar-body"></div><div id="avatar-status-label">Idle</div></div></div>
        <div id="chat-wrapper"><div id="chat-container"><div id="chat-header"><button id="toggle-voice-btn" class="header-btn" title="Toggle Voice Output">🔊</button><span id="header-title">Mindful Companion ({user['username']})</span><button id="fullscreen-btn" class="header-btn" title="Toggle Fullscreen">⛶</button></div><div id="chat-history"></div><div id="status-indicator"></div><div id="input-area"><button id="mic-btn" title="Click to Speak">🎤</button><input type="text" id="user-input" placeholder="Type or click the mic to speak..."><button id="send-button">Send</button></div></div></div>
    </div>
    <script>
        const chatHistory = document.getElementById('chat-history'); const userInput = document.getElementById('user-input'); const sendButton = document.getElementById('send-button'); const micBtn = document.getElementById('mic-btn'); const toggleVoiceBtn = document.getElementById('toggle-voice-btn'); const statusIndicator = document.getElementById('status-indicator'); const fullscreenBtn = document.getElementById('fullscreen-btn'); const mainLayout = document.getElementById('main-layout'); const avatarContainer = document.getElementById('avatar-container'); const statusLabel = document.getElementById('avatar-status-label'); const mouthBars = document.querySelectorAll('#avatar-mouth .bar');
        let isVoiceEnabled = true; let isFullscreen = false; let isRecording = false;
        let audioState = {{ unlocked: false, isPlaying: false, currentAudio: null, initialData: {json.dumps(initial_audio_b64)} }};
        let initialGreetingText = {json.dumps(greeting_message)};
        let mouthAnimationInterval;
        let recognition = null;

        function setAvatarState(state) {{
            avatarContainer.className = state;
            statusLabel.textContent = state.charAt(0).toUpperCase() + state.slice(1);
            clearInterval(mouthAnimationInterval);
            if (state === 'speaking') {{
                mouthAnimationInterval = setInterval(() => {{
                    mouthBars.forEach(bar => {{
                        bar.style.height = `${{Math.random() * 12 + 4}}px`;
                    }});
                }}, 150);
            }} else {{
                mouthBars.forEach(bar => {{
                    bar.style.height = '';
                }});
            }}
        }}

        function appendMessage(text, sender) {{
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', sender + '-message');
            messageDiv.textContent = text;
            chatHistory.appendChild(messageDiv);
            chatHistory.scrollTop = chatHistory.scrollHeight;
        }}

        function stopCurrentAudio() {{
            if (audioState.currentAudio) {{
                audioState.currentAudio.pause();
                audioState.currentAudio.currentTime = 0;
                audioState.isPlaying = false;
            }}
        }}

        function playAudio(base64Data) {{
            stopCurrentAudio();
            if (!isVoiceEnabled || !base64Data) {{
                setAvatarState('idle');
                return;
            }}
            setAvatarState('speaking');
            const audioData = "data:audio/mpeg;base64," + base64Data;
            audioState.currentAudio = new Audio(audioData);
            audioState.isPlaying = true;
            audioState.currentAudio.play().catch(e => {{
                console.error("Audio play failed:", e);
                audioState.isPlaying = false;
                setAvatarState('idle');
            }});
            audioState.currentAudio.onended = () => {{
                audioState.isPlaying = false;
                setAvatarState('idle');
            }};
        }}

        function unlockAudioContext() {{
            if (audioState.unlocked) return;
            audioState.unlocked = true;
            console.log("Audio context unlocked.");
            playAudio(audioState.initialData);
            audioState.initialData = null;
        }}

        function sendMessage() {{
            unlockAudioContext();
            stopCurrentAudio();
            const prompt = userInput.value.trim();
            if (prompt === "") return;
            appendMessage(prompt, 'user');
            userInput.value = "";
            setAvatarState('thinking');
            statusIndicator.textContent = "Companion is thinking...";
            sendButton.disabled = true;
            micBtn.disabled = true;
            google.colab.kernel.invokeFunction('process_prompt_from_ui', [prompt], {{}});
        }}

        window.displayBotMessage = (data) => {{
            appendMessage(data.text, 'bot');
            statusIndicator.textContent = "";
            sendButton.disabled = false;
            micBtn.disabled = false;
            userInput.focus();
            if (audioState.unlocked) {{
                playAudio(data.audio);
            }} else {{
                setAvatarState('idle');
            }}
        }};

        // Initialize Speech Recognition with better error handling
        const SpeechRecognition = window.SpeechRecognition || window.webkitSpeechRecognition;
        if (SpeechRecognition) {{
            try {{
                recognition = new SpeechRecognition();
                recognition.continuous = false;
                recognition.interimResults = false;
                recognition.lang = 'en-US';

                micBtn.addEventListener('click', () => {{
                    if (isRecording) {{
                        // Stop recording if already recording
                        recognition.stop();
                        isRecording = false;
                        micBtn.classList.remove('recording');
                        setAvatarState('idle');
                        statusIndicator.textContent = "";
                        return;
                    }}

                    unlockAudioContext();
                    stopCurrentAudio();

                    try {{
                        recognition.start();
                        isRecording = true;
                        micBtn.classList.add('recording');
                    }} catch(e) {{
                        console.error("Recognition start error:", e);
                        statusIndicator.textContent = "Please wait, microphone is busy...";
                        setTimeout(() => {{ statusIndicator.textContent = ""; }}, 2000);
                    }}
                }});

                recognition.onstart = () => {{
                    isRecording = true;
                    micBtn.classList.add('recording');
                    setAvatarState('listening');
                    statusIndicator.textContent = "Listening... (click mic to stop)";
                }};

                recognition.onresult = (event) => {{
                    const transcript = event.results[0][0].transcript;
                    userInput.value = transcript;
                    sendMessage();
                }};

                recognition.onerror = (event) => {{
                    isRecording = false;
                    micBtn.classList.remove('recording');
                    console.error("Speech recognition error:", event.error);

                    let errorMessage = "";
                    switch(event.error) {{
                        case 'not-allowed':
                        case 'permission-denied':
                            errorMessage = "⚠️ Microphone access denied. Please allow microphone access in your browser settings.";
                            // Check if running in iframe
                            if (window.self !== window.top) {{
                                errorMessage = "⚠️ Mic blocked in iframe. Click the 'Pop out' button (top-right) to open in a new window.";
                            }}
                            break;
                        case 'no-speech':
                            errorMessage = "No speech detected. Please try again.";
                            break;
                        case 'audio-capture':
                            errorMessage = "No microphone found. Please check your device.";
                            break;
                        case 'network':
                            errorMessage = "Network error. Please check your connection.";
                            break;
                        case 'aborted':
                            // User manually stopped, don't show error
                            break;
                        default:
                            errorMessage = `Microphone error: ${{event.error}}`;
                    }}

                    if (errorMessage) {{
                        setAvatarState('idle');
                        statusIndicator.textContent = errorMessage;
                        setTimeout(() => {{ statusIndicator.textContent = ""; }}, 5000);
                    }}
                }};

                recognition.onend = () => {{
                    isRecording = false;
                    micBtn.classList.remove('recording');
                    if (avatarContainer.className === 'listening') {{
                        setAvatarState('idle');
                        if (statusIndicator.textContent === "Listening... (click mic to stop)") {{
                            statusIndicator.textContent = "";
                        }}
                    }}
                }};

            }} catch(e) {{
                console.error("Speech recognition initialization error:", e);
                micBtn.disabled = true;
                micBtn.style.color = '#ccc';
                micBtn.title = 'Voice recognition initialization failed.';
            }}
        }} else {{
            micBtn.disabled = true;
            micBtn.style.color = '#ccc';
            micBtn.title = 'Voice recognition not supported. Please use Chrome or Edge.';
        }}

        toggleVoiceBtn.addEventListener('click', () => {{
            isVoiceEnabled = !isVoiceEnabled;
            toggleVoiceBtn.textContent = isVoiceEnabled ? '🔊' : '🔇';
            if (!isVoiceEnabled) {{
                stopCurrentAudio();
                setAvatarState('idle');
            }}
        }});

        function toggleFullscreen() {{
            if (!isFullscreen) {{
                const overlay = document.createElement('div');
                overlay.id = 'fullscreen-overlay';
                overlay.classList.add('fullscreen-overlay');
                mainLayout.classList.add('fullscreen-chat-layout');
                overlay.appendChild(mainLayout);
                document.body.appendChild(overlay);
                isFullscreen = true;
            }} else {{
                const overlay = document.getElementById('fullscreen-overlay');
                const originalWrapper = document.querySelector('.cell-output-wrapper') || document.body;
                originalWrapper.appendChild(mainLayout);
                if (overlay && overlay.parentNode) {{
                    document.body.removeChild(overlay);
                }}
                mainLayout.classList.remove('fullscreen-chat-layout');
                isFullscreen = false;
            }}
            userInput.focus();
        }}

        sendButton.addEventListener('click', sendMessage);
        userInput.addEventListener('keydown', (e) => {{ if (e.key === 'Enter') sendMessage(); }});
        fullscreenBtn.addEventListener('click', toggleFullscreen);

        appendMessage(initialGreetingText, 'bot');
        setAvatarState('idle');
        userInput.focus();
    </script>
    """
    display(HTML(ui_html))
    ui_loaded = True

# --- Start the Application ---
# Define the global variable before the function that uses it is called.
ui_loaded = False
start_app()

🤖 Mindful Companion: Welcome!
To get started, what's your name? Nipun


In [ ]:
# @title Step 5: Run the Companion with Full UI, Voice, and Smart Greeting

import json
from IPython.display import display, HTML
from google.colab import output
import base64

# --- Voice Generation Function ---
def generate_audio_base64(text, voice_id="21m00Tcm4TlvDq8ikWAM"): # Voice: "Rachel"
    if not ELEVENLABS_API_KEY:
        print("⚠️ ElevenLabs API key not configured. Skipping audio generation.")
        return None
    TTS_URL = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {"Accept": "audio/mpeg", "Content-Type": "application/json", "xi-api-key": ELEVENLABS_API_KEY}
    data = {"text": text, "model_id": "eleven_multilingual_v2", "voice_settings": {"stability": 0.5, "similarity_boost": 0.75}}
    try:
        response = requests.post(TTS_URL, json=data, headers=headers)
        response.raise_for_status()
        return base64.b64encode(response.content).decode('utf-8')
    except requests.exceptions.RequestException as e:
        print(f"Error during ElevenLabs API call: {e}")
        return None

# --- Python Backend Logic for UI ---
def process_prompt_from_ui(prompt):
    global user
    response_text = ""
    # Command Handling
    if prompt.lower().startswith('/'):
        if prompt.lower() == '/view':
            import io; from contextlib import redirect_stdout
            f = io.StringIO();
            with redirect_stdout(f): view_tasks(user['id'])
            response_text = f.getvalue()
        else: # Handle other commands briefly
            response_text = "Command executed."
    else: # Mindful Conversation
        conversation_history = get_recent_conversations(user['id'], limit=30)
        memories = retrieve_relevant_memories(user['id'], prompt, top_k=7)
        ai_output = get_ai_response(prompt, user, conversation_history, memories)
        response_text = ai_output.get('response', "I'm not sure how to reply.")
        # Database updates
        new_summary = ai_output.get('updated_summary', user['personality_summary'])
        new_behavioral_notes = ai_output.get('updated_behavioral_notes', user.get('behavioral_notes'))
        log_conversation(user['id'], 'user', prompt)
        log_conversation(user['id'], 'model', response_text)
        if new_summary != user['personality_summary']:
            update_user(user['id'], "personality_summary", new_summary); user['personality_summary'] = new_summary
        if new_behavioral_notes != user.get('behavioral_notes'):
            update_user(user['id'], "behavioral_notes", new_behavioral_notes); user['behavioral_notes'] = new_behavioral_notes

    audio_b64 = generate_audio_base64(response_text)
    response_data = json.dumps({"text": response_text, "audio": audio_b64})
    output.eval_js(f'window.displayBotMessage({response_data})')

# Register the Python function so JavaScript can call it
output.register_callback('process_prompt_from_ui', process_prompt_from_ui)


# --- Main Application Start ---
def start_app():
    if 'user' in globals(): # Prevent re-running setup
        print("UI already loaded. If you want to restart, run Runtime > Restart session.")
        return

    init_db()
    print("🤖 Mindful Companion: Welcome!")
    username = input("To get started, what's your name? ")
    global user
    user = get_or_create_user(username)

    # --- Smart Greeting Logic ---
    greeting_message = ""
    now = datetime.datetime.now()
    if user['last_seen']:
        last_seen_dt = datetime.datetime.fromisoformat(user['last_seen'])
        time_since = now - last_seen_dt
        if time_since.days >= 1:
            # Generate a proactive check-in if it's been a while
            checkin_prompt = f"You are checking in with your friend {username}, who you haven't spoken to in over a day. Your last understanding of them was: \"{user['personality_summary']}\". Generate a single, gentle, short, and friendly check-in message."
            checkin_response = chat_model.generate_content(checkin_prompt)
            greeting_message = checkin_response.text
        else:
            greeting_message = f"Hello again, {username}! What's on your mind today?"
    else:
        greeting_message = f"Hello, {username}! It's nice to meet you. How can I help you today?"

    update_user(user['id'], "last_seen", now.isoformat())

    # Generate audio for the initial greeting
    initial_audio_b64 = generate_audio_base64(greeting_message)

    # --- HTML, CSS, and JavaScript for the UI ---
    ui_html = f"""
    <style>
        /* All your previous CSS styles for the UI go here */
        @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;500&display=swap');
        #ui-wrapper {{ display: flex; justify-content: center; align-items: center; width: 100%; height: 85vh; }}
        #chat-container {{ width: 100%; max-width: 700px; height: 100%; max-height: 600px; border: 1px solid #ccc; border-radius: 10px; display: flex; flex-direction: column; font-family: 'Roboto', sans-serif; background-color: #f9f9f9; box-shadow: 0 4px 12px rgba(0,0,0,0.1); transition: all 0.3s ease-in-out; }}
        #chat-header {{ position: relative; padding: 15px; background: linear-gradient(135deg, #6e8efb, #a777e3); color: white; font-size: 18px; font-weight: 500; border-top-left-radius: 10px; border-top-right-radius: 10px; text-align: center; display: flex; justify-content: center; align-items: center; }}
        #fullscreen-btn {{ position: absolute; right: 15px; top: 50%; transform: translateY(-50%); background: none; border: 1px solid rgba(255,255,255,0.5); color: white; font-size: 16px; cursor: pointer; border-radius: 5px; width: 30px; height: 30px; line-height: 28px; padding: 0; transition: background-color 0.2s; }}
        #fullscreen-btn:hover {{ background-color: rgba(255,255,255,0.2); }}
        #chat-history {{ flex-grow: 1; overflow-y: auto; padding: 20px; display: flex; flex-direction: column; gap: 15px; }}
        .message {{ max-width: 80%; padding: 12px 18px; border-radius: 20px; line-height: 1.5; word-wrap: break-word; white-space: pre-wrap; }}
        .user-message {{ align-self: flex-end; background-color: #0b93f6; color: white; border-bottom-right-radius: 5px; }}
        .bot-message {{ align-self: flex-start; background-color: #e5e5ea; color: black; border-bottom-left-radius: 5px; }}
        #input-area {{ display: flex; padding: 15px; border-top: 1px solid #ccc; }}
        #user-input {{ flex-grow: 1; border: 1px solid #ddd; border-radius: 20px; padding: 10px 15px; font-size: 16px; outline: none; transition: border-color 0.2s; }}
        #user-input:focus {{ border-color: #6e8efb; }}
        #send-button {{ margin-left: 10px; padding: 10px 20px; border: none; background-color: #6e8efb; color: white; border-radius: 20px; cursor: pointer; font-size: 16px; transition: background-color 0.2s; }}
        #send-button:hover {{ background-color: #5a78d1; }}
        #status-indicator {{ padding: 5px 20px; font-style: italic; color: #888; height: 25px; }}
        .fullscreen-overlay {{ position: fixed; top: 0; left: 0; width: 100vw; height: 100vh; background-color: rgba(0,0,0,0.7); display: flex; justify-content: center; align-items: center; z-index: 9999; }}
        .fullscreen-chat {{ width: 95vw; height: 95vh; max-width: 1400px; max-height: 95vh; }}
    </style>

    <div id="ui-wrapper">
        <div id="chat-container">
            <div id="chat-header">
                <span>Mindful Companion ({user['username']})</span>
                <button id="fullscreen-btn" title="Toggle Fullscreen">[⛶]</button>
            </div>
            <div id="chat-history"></div>
            <div id="status-indicator"></div>
            <div id="input-area">
                <input type="text" id="user-input" placeholder="Type your message or a command...">
                <button id="send-button">Send</button>
            </div>
        </div>
    </div>

    <script>
        const chatContainer = document.getElementById('chat-container');
        const chatHistory = document.getElementById('chat-history');
        const userInput = document.getElementById('user-input');
        const sendButton = document.getElementById('send-button');
        const statusIndicator = document.getElementById('status-indicator');
        const fullscreenBtn = document.getElementById('fullscreen-btn');
        const uiWrapper = document.getElementById('ui-wrapper');
        let currentAudio = null;
        let isFullscreen = false;

        function appendMessage(text, sender) {{
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', sender + '-message');
            messageDiv.textContent = text;
            chatHistory.appendChild(messageDiv);
            chatHistory.scrollTop = chatHistory.scrollHeight;
        }}

        function sendMessage() {{
            if (currentAudio) {{ currentAudio.pause(); currentAudio.currentTime = 0; }}
            const prompt = userInput.value.trim();
            if (prompt === "") return;
            appendMessage(prompt, 'user');
            userInput.value = "";
            statusIndicator.textContent = "Companion is thinking...";
            sendButton.disabled = true;
            google.colab.kernel.invokeFunction('process_prompt_from_ui', [prompt], {{}});
        }}

        window.displayBotMessage = (data) => {{
            appendMessage(data.text, 'bot');
            statusIndicator.textContent = "";
            sendButton.disabled = false;
            userInput.focus();
            if (data.audio) {{
                const audioData = "data:audio/mpeg;base64," + data.audio;
                currentAudio = new Audio(audioData);
                currentAudio.play();
            }}
        }}

        function toggleFullscreen() {{ /* ... your existing fullscreen logic ... */ }}

        sendButton.addEventListener('click', sendMessage);
        userInput.addEventListener('keydown', (event) => {{ if (event.key === 'Enter') sendMessage(); }});
        fullscreenBtn.addEventListener('click', toggleFullscreen);

        // --- New, Simplified Greeting ---
        // We use json.dumps to safely pass the string to JavaScript
        const initialGreetingText = {json.dumps(greeting_message)};
        const initialAudioData = {json.dumps(initial_audio_b64)};

        appendMessage(initialGreetingText, 'bot');
        if (initialAudioData) {{
            const audioData = "data:audio/mpeg;base64," + initialAudioData;
            currentAudio = new Audio(audioData);
            currentAudio.play();
        }}
    </script>
    """

    # CRITICAL: This line displays the UI. It was missing before.
    display(HTML(ui_html))

# --- Start the application ---
start_app()

🤖 Mindful Companion: Welcome!
To get started, what's your name? Emran
